# Chat With RAG

### Load dir with PDFs

In [1]:
from langchain.document_loaders import DirectoryLoader

In [2]:
loader = DirectoryLoader('data', glob="*.pdf")
documents = loader.load()

/Users/shiraelitzur/python_environments/rag_env123/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Split the docs into chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

In [7]:
# chunks[5]
chunks[0]

Document(metadata={'source': 'data/friendsS1.pdf', 'start_index': 0}, page_content="The One Where Monica Gets a New Roommate (The Pilot-The Uncut Version)\n\nWritten by: Marta Kauffman & David Crane Transcribed by: guineapig Additional transcribing by: Eric Aasen (Note: The previously unseen parts of this episode are shown in blue text.)\n\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\n\nMonica: There's nothing to tell! He's just some guy I work with!\n\nJoey: C'mon, you're going out with the guy! There's gotta be something wrong with him!\n\nChandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?\n\nPhoebe: Wait, does he eat chalk?\n\n(They all stare, bemused.)\n\nPhoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!\n\nMonica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.\n\nChandler: Sounds like a date to me.\n\n[Time Lapse]")

In [10]:
print(chunks[-1].metadata)

{'source': 'data/harry_potter1.pdf', 'start_index': 456855}


### Create an embeddings function and examine embeddings

In [11]:
from langchain.embeddings import GPT4AllEmbeddings

In [12]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embedding_function = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

In [ ]:
vector = embedding_function.embed_query("apple")
vector

In [20]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embedding_function)
x = evaluator.evaluate_string_pairs(prediction="apple", prediction_b="golan")
print(x)

{'score': 0.8075914250086906}


### Create / Load the vector store

In [28]:
from langchain.vectorstores.chroma import Chroma

In [26]:
CHROMA_PATH = "chroma_db_friends"

db = Chroma.from_documents(
    chunks, embedding_function, persist_directory=CHROMA_PATH
)
# db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [30]:
query = "What is the name of the coffee shop?"
results = db.similarity_search_with_relevance_scores(query, k=3)
print(results[0])

(Document(metadata={'source': 'data/friendsS1.pdf', 'start_index': 23940}, page_content="Phoebe: (sings) What I said you had...\n\nMonica: (to Phoebe) Would you stop?\n\nPhoebe: Oh, was I doing it again?\n\nAll: Yes!\n\nMonica: I said that you had a nice butt, it's just not a great butt.\n\nJoey: Oh, you wouldn't know a great butt if it came up and bit ya.\n\nRoss: There's an image.\n\nRachel: (walks up with a pot of coffee) Would anybody like more coffee?\n\nChandler: Did you make it, or are you just serving it?\n\nRachel: I'm just serving it.\n\nAll: Yeah. Yeah, I'll have a cup of coffee.\n\nChandler: Kids, new dream... I'm in Las Vegas. (Rachel sits down to hear Chandler's dream.)\n\nCustomer: (To Rachel) Ahh, miss? More coffee?\n\nRachel: Ugh. (To another customer that's leaving.) Excuse me, could you give this to that guy over there? (Hands him the coffee pot.) Go ahead. (He does so.) Thank you. (To the gang.) Sorry. Okay, Las Vegas.\n\nChandler: Okay, so, I'm in Las Vegas... I'm 

In [39]:
# query = "Describe Prof. McGonagall"
# results = db.similarity_search_with_relevance_scores(query, k=3)
# print(results)

## Use the retrieved document as context for the LLM

In [31]:
from langchain.prompts import ChatPromptTemplate

In [32]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context.
If not provided or not relevant, answer 'I do not have enough information to answer
that question'. the context:
{context}
------
Answer the question based on the above context: {query}
"""

In [ ]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, query=query)
print(prompt)

In [34]:
from langchain_community.chat_models import ChatOllama

In [37]:
llm = ChatOllama(model='mistral', format="json", temperature=0)
response_text = llm.invoke(prompt)

In [38]:
response_text

AIMessage(content='{ "Central Perk" \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', id='run-e18eb618-bed5-4373-9c81-84e54b8470d5-0')